In [60]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [61]:
import statsmodels.api as sm
import statsmodels.formula.api as sf

In [62]:
# 서열척도
col1=[['(가)건강 만족도', '(다)주거 환경 만족도'], ['(나)가족의 수입 만족도', '교육수준'],
       ['(다)주거 환경 만족도', '교육수준'], ['(라)가족관계 만족도', '교육수준'],
       ['(마)직업 만족도', '교육수준'], ['(바)사회적 친분관계 만족도', '교육수준'],
       ['(사)여가생활 만족도', '교육수준']]
# 카이제곱
col2=[['(다)주거 환경 만족도', '교육수준'],['(다)주거 환경 만족도', '종교'],
       ['(다)주거 환경 만족도', '만성질환'],['(다)주거 환경 만족도', '근로유형']]
# 범주~연속형(anova)
col3=[['(나)가족의 수입 만족도', '방수'],['(다)주거 환경 만족도', '연령'],
       ['(마)직업 만족도', '방수'],['(바)사회적 친분관계 만족도', '방수'],
       ['(사)여가생활 만족도', '방수']]
# 연속형~연속형(regression)
col4=[['방수','연령'],['연령','주거면적'],['총생활비','방수'],['주거면적','총생활비'],['총생활비','주거면적']]
c1=list(sum(list(map(list,col1)),[]))
c1.extend(list(set(sum(list(map(list,col2)),[]))))
c2=list(sum(list(map(list,col3)),[]))
c3=list(sum(list(map(list,col4)),[]))

In [63]:
D=pd.read_excel("../02_데이터검정/카테고리_소분류완료.xlsx",usecols=list(set(c1+c2+c3))+["삶의 사다리 점수"])
data=pd.DataFrame(D)
# 명목형의 데이터 형태 바꾸기
for col in ["근로유형","종교","만성질환"]:
    data[col]=data[col].astype("category")

## train,test 데이터 분리

In [64]:
train_x,test_x,train_y,test_y=train_test_split(data[data.columns.difference(["삶의 사다리 점수"])],data["삶의 사다리 점수"],train_size=0.7,stratify=data["삶의 사다리 점수"],random_state=10)

In [65]:
def Mylogit(y,x):
    logit_dc={}
    for i in y.unique():
        # 예측해야하는 값은 1 아니면 0
        use_y=y.apply(lambda k:1 if k==i else 0)         
        use_x=pd.get_dummies(sm.add_constant(x),drop_first=True,dtype="int")
        model=sm.Logit(exog=use_x,endog=use_y).fit()
        logit_dc[i]=model
    return logit_dc

In [66]:
col=[x for x in data.columns if x.find("만족도")!=-1]

In [67]:
# #컬럼 선택 후 학습
# test=Mylogit(train_y,train_x[col]) #우리의 소프트맥스 함수(역할군)
# df_data={}
# for i in test.keys():
#     df_data[i]=test[i].predict(pd.get_dummies(sm.add_constant(train_x[col]),drop_first=True,dtype="int"))
# #학습결과표
# result_df=pd.DataFrame(df_data)
# result_softmax=result_df.apply(sum,axis=1)
# result_df["max"]=result_df.apply(max,axis=1)
# for i in result_df.index:
#     for col in result_df.columns:
#         if result_df.loc[i,col]==result_df.loc[i,'max']:
#             result_df.loc[i,'predict_value']=col
#             break
# result_df["softmax"]=result_df["max"]/result_softmax
# #학습결과 -> 정확도 추출
# score_df=pd.merge(result_df,train_y,right_index=True,left_index=True)[["predict_value","삶의 사다리 점수","softmax"]]
# score_df["yes/no"]=score_df['predict_value']==score_df['삶의 사다리 점수']
# Accuracy=score_df["yes/no"].sum()/2333

In [68]:
# 궁금하니까 해보는거임

In [69]:
import itertools

In [70]:
col1.extend(col2)

In [71]:
for x in [col1,col3,col4]:
    x.append([])

In [72]:
sum_col=[sum(y,[]) for y in [x for x in itertools.product(col1,col3,col4)] if any(y)]

In [73]:
comb=[]
for i in range(1,len(data.columns.difference(["삶의 사다리 점수"]))+1):
    comb.extend([x for x in itertools.combinations(data.columns.difference(["삶의 사다리 점수"]),r=i)])
comb=list(map(lambda x:[y for y in x if y],comb))

In [74]:
final_result=[]
for col_name in sum_col:
    try:
        test=Mylogit(train_y,train_x[col_name]) #우리의 소프트맥스 함수(역할군)
        df_data={}
        for i in test.keys():
            df_data[i]=test[i].predict(pd.get_dummies(sm.add_constant(train_x[col_name]),drop_first=True,dtype="int"))
        #학습결과표
        result_df=pd.DataFrame(df_data)
        result_softmax=result_df.apply(sum,axis=1)
        result_df["max"]=result_df.apply(max,axis=1)
        for i in result_df.index:
            for col in result_df.columns:
                if result_df.loc[i,col]==result_df.loc[i,'max']:
                    result_df.loc[i,'predict_value']=col
                    break
        result_df["softmax"]=result_df["max"]/result_softmax
        #학습결과 -> 정확도 추출
        score_df=pd.merge(result_df,train_y,right_index=True,left_index=True)[["predict_value","삶의 사다리 점수","softmax"]]
        score_df["yes/no"]=score_df['predict_value']==score_df['삶의 사다리 점수']
        Accuracy=score_df["yes/no"].sum()/len(train_y)
        final_result.append({'Model':col_name,"Accuracy":Accuracy})
    except:
        continue

Optimization terminated successfully.
         Current function value: 0.396233
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.469464
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.395394
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.469081
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.034058
         Iterations 11
Optimization terminated successfully.
         Current function value: 0.506859
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.271998
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.480640
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.189829
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.181406
 

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.407843
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.407141
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.472052
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.033370
         Iterations 11
Optimization terminated successfully.
         Current function value: 0.506927
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.274500
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.485833
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.190950
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.187840
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.129051
 

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.407141
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.472052
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.033370
         Iterations 11
Optimization terminated successfully.
         Current function value: 0.506927
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.274500
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.485833
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.190950
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.187840
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.129051
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.078567
 

...

Optimization terminated successfully.
         Current function value: 0.412403
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.472102
         Iterations 6
         Current function value: 0.038573
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.505183
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.278058
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.484692
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.192362
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.199574
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.131683
         Iterations 8
         Current function value: 0.078755
         Iterations: 35
         Current function value: 0.008739
         I

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.414518
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.473297
         Iterations 6
         Current function value: 0.042399
         Iterations: 35
Optimization terminated successfully.
         Current function value: 0.505540
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.281712
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.485420
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.193861
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.203520
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.132600
         Iterations 8
         Current function value: 0.081242
         Iterations: 35
         Current function value: 0.008916
         I

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 0.412509
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.410885
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.473421
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.036678
         Iterations 10
Optimization terminated successfully.
         Current function value: 0.507616
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.280385
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.484533
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.193159
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.192955
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.130825
 

In [75]:
use_col_name=[y for y in final_result if y["Accuracy"]==max([x["Accuracy"] for x in final_result])]

In [76]:
cols_list=[x["Model"] for x in use_col_name]

In [77]:
cols_list.append(['(다)주거 환경 만족도', '교육수준'])
cols_list.append(['(다)주거 환경 만족도', '교육수준',"연령"])
cols_list.append(['(다)주거 환경 만족도', '교육수준',"교육수준"])
cols_list.append(['(다)주거 환경 만족도', '교육수준',"연령","교육수준"])

In [82]:
cols_list

[['(가)건강 만족도', '(다)주거 환경 만족도', '(사)여가생활 만족도', '방수', '주거면적', '총생활비'],
 ['(가)건강 만족도', '(다)주거 환경 만족도', '(사)여가생활 만족도', '방수', '총생활비', '주거면적'],
 ['(다)주거 환경 만족도', '교육수준'],
 ['(다)주거 환경 만족도', '교육수준', '연령'],
 ['(다)주거 환경 만족도', '교육수준', '교육수준'],
 ['(다)주거 환경 만족도', '교육수준', '연령', '교육수준']]

In [83]:
pred_result=[]
for col_name in cols_list:
    try:
        test=Mylogit(train_y,train_x[col_name]) #우리의 소프트맥스 함수(역할군)
        df_data={}
        for i in test.keys():
            df_data[i]=test[i].predict(pd.get_dummies(sm.add_constant(test_x[col_name]),drop_first=True,dtype="int"))
        #학습결과표
        result_df=pd.DataFrame(df_data)
        result_softmax=result_df.apply(sum,axis=1)
        result_df["max"]=result_df.apply(max,axis=1)
        for i in result_df.index:
            for col in result_df.columns:
                if result_df.loc[i,col]==result_df.loc[i,'max']:
                    result_df.loc[i,'predict_value']=col
                    break
        result_df["softmax"]=result_df["max"]/result_softmax
        #학습결과 -> 정확도 추출
        score_df=pd.merge(result_df,test_y,right_index=True,left_index=True)[["predict_value","삶의 사다리 점수","softmax"]]
        score_df["yes/no"]=score_df['predict_value']==score_df['삶의 사다리 점수']
        Accuracy=score_df["yes/no"].sum()/len(train_y)
        pred_result.append({'Model':col_name,"Pred_Accuracy":Accuracy})
    except:
        continue

Optimization terminated successfully.
         Current function value: 0.395404
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.462446
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.037292
         Iterations 11
Optimization terminated successfully.
         Current function value: 0.506022
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.269571
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.481146
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.184526
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.180882
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.129041
         Iterations 8
Optimization terminated successfully.
         Current function value: 0.069574
 

In [84]:
pred_result

[{'Model': ['(가)건강 만족도', '(다)주거 환경 만족도', '(사)여가생활 만족도', '방수', '주거면적', '총생활비'],
  'Pred_Accuracy': 0.12387483926275182},
 {'Model': ['(가)건강 만족도', '(다)주거 환경 만족도', '(사)여가생활 만족도', '방수', '총생활비', '주거면적'],
  'Pred_Accuracy': 0.12387483926275182},
 {'Model': ['(다)주거 환경 만족도', '교육수준'], 'Pred_Accuracy': 0.10715816545220745},
 {'Model': ['(다)주거 환경 만족도', '교육수준', '연령'],
  'Pred_Accuracy': 0.11530218602657523}]

In [80]:
test_x

,(가)건강 만족도,(나)가족의 수입 만족도,(다)주거 환경 만족도,(라)가족관계 만족도,(마)직업 만족도,(바)사회적 친분관계 만족도,(사)여가생활 만족도,교육수준,근로유형,만성질환,방수,연령,종교,주거면적,총생활비
2273,2,3,3,3,3,3,3,2,4,1,3,85,0,96,65
3011,5,4,4,4,4,4,3,7,1,0,2,32,0,50,191
511,3,2,4,4,2,4,4,6,1,0,1,43,1,26,388
1129,2,4,4,5,3,4,4,5,4,1,3,91,0,66,83
1002,2,2,4,2,2,3,3,3,4,1,2,86,0,50,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,2,2,4,4,4,4,4,7,1,0,3,43,0,82,588
2055,3,4,4,3,3,3,3,3,4,1,3,85,0,66,50
2790,2,2,3,2,3,2,2,2,4,1,1,74,0,20,79
146,4,4,4,4,4,4,3,7,1,0,3,33,1,79,227
